In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-05cdaa7b-9dc5-817f-752d-e4c66e1af08a)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/CHEXPERT/CheXpert-v1.0-small.zip > /dev/null

In [ ]:
!pip install timm > /dev/null

In [ ]:
import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import torch 
from torch import nn
from torchsummary import summary

import timm
from torch.optim import Adam, lr_scheduler

from myGen import MyGen

from sklearn.metrics import roc_auc_score

In [ ]:
train = pd.read_csv('/content/CheXpert-v1.0-small/train.csv')

In [ ]:
train.loc[:, train.columns[5:]] = train.loc[:, train.columns[5:]].fillna(0)

zeros = ['Cardiomegaly', 'Consolidation', 'Pleural Effusion']
train.loc[:, zeros] = train.loc[:, zeros].replace({-1:0})
ones = ['Edema', 'Atelectasis']
train.loc[:, ones] = train.loc[:, ones].replace({-1:1})

train = train[['Path', 'Frontal/Lateral', 'Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']].copy() 

In [ ]:
train.to_csv('/content/CheXpert-v1.0-small/train_edited.csv', index=False)

In [ ]:
save_weights_path = '/content/drive/MyDrive/Final_Weights/Z_efficientnet_b2_320.pth'
load_weights_path = '/content/drive/MyDrive/Final_Weights/Z_efficientnet_b2_320_load.pth'

In [ ]:
timm.list_models(pretrained=True)

In [ ]:
model = timm.create_model('efficientnet_b2', num_classes=5, pretrained=True)
model = model.cuda()

# summary(model, (3, 224, 224), batch_size=1, device='cuda')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b2_ra-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_ra-bcdf34b7.pth


In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
root = '/content/'

trainGen = MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/train_edited.csv', 
                 image_size=320, 
                 frontal=True)

valGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=320, 
                 frontal=True)

trainloader =  torch.utils.data.DataLoader(trainGen, batch_size=32, shuffle=True)
testloader =  torch.utils.data.DataLoader(valGen, batch_size=1, shuffle=False)

# define loss & optimizer
Loss = nn.BCELoss()

optimizer = Adam(model.parameters(), lr=1e-3)

scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Decay LR by a factor of 0.5 every 5 epochs

#model.load_state_dict(torch.load(load_weights_path))

# training
const_threshold = 5

val_auc_mean = 0
best_val_auc = 0
const_cnt = 0

for epoch in range(5):

    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 800 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for data in testloader:
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  y_pred = torch.sigmoid(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())            
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              model.train()

              if best_val_auc < val_auc_mean:
                  const_cnt = 0
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  best_val_auc = val_auc_mean
                  torch.save(model.state_dict(), save_weights_path)
              else:
                  const_cnt += 1
                  scheduler.step()            

              if (const_cnt > const_threshold):
                  const_cnt = 0 
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  model.load_state_dict(torch.load(save_weights_path))
                               
              lr = get_lr(optimizer)

              print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f, lr=%f' %(epoch, idx, val_auc_mean, best_val_auc, lr))

Epoch=0, BatchID=0, Val_AUC=0.5123, Best_Val_AUC=0.5123, lr=0.001000
Epoch=0, BatchID=800, Val_AUC=0.8856, Best_Val_AUC=0.8856, lr=0.001000
Epoch=0, BatchID=1600, Val_AUC=0.8650, Best_Val_AUC=0.8856, lr=0.001000
Epoch=0, BatchID=2400, Val_AUC=0.8706, Best_Val_AUC=0.8856, lr=0.001000
Epoch=0, BatchID=3200, Val_AUC=0.8820, Best_Val_AUC=0.8856, lr=0.001000
Epoch=0, BatchID=4000, Val_AUC=0.8747, Best_Val_AUC=0.8856, lr=0.001000
Epoch=0, BatchID=4800, Val_AUC=0.8820, Best_Val_AUC=0.8856, lr=0.000500
Epoch=0, BatchID=5600, Val_AUC=0.8893, Best_Val_AUC=0.8893, lr=0.000500
Epoch=1, BatchID=0, Val_AUC=0.8918, Best_Val_AUC=0.8918, lr=0.000500
Epoch=1, BatchID=800, Val_AUC=0.8778, Best_Val_AUC=0.8918, lr=0.000500
Epoch=1, BatchID=1600, Val_AUC=0.8789, Best_Val_AUC=0.8918, lr=0.000500
Epoch=1, BatchID=2400, Val_AUC=0.8911, Best_Val_AUC=0.8918, lr=0.000500
Epoch=1, BatchID=3200, Val_AUC=0.8924, Best_Val_AUC=0.8924, lr=0.000500
Epoch=1, BatchID=4000, Val_AUC=0.8967, Best_Val_AUC=0.8967, lr=0.000500


In [ ]:
root = '/content/'
testGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=224, 
                 frontal=True)

testloader =  torch.utils.data.DataLoader(testGen, batch_size=1, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(save_weights_path))

<All keys matched successfully>

In [ ]:
model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for data in testloader:
        test_data, test_labels = data
        test_data = test_data.cuda()
        y_pred = model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

In [ ]:
labels = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']

results = pd.DataFrame(index=labels)


scores = []
for i in range(5):
    score = roc_auc_score(test_true[:, i], test_pred[:, i])
    scores.append(score)
    
results['AUC'] = scores

In [ ]:
results

,AUC
Cardiomegaly,0.876671
Edema,0.936905
Consolidation,0.940993
Atelectasis,0.860262
Pleural Effusion,0.917799


In [ ]:
results['AUC'].mean()

0.9065259824386634